In [1]:
%pip install langchain-google-genai langchain-core pytz python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage # [수정] ToolMessage 추가
from langchain_core.tools import tool
from datetime import datetime
import pytz

load_dotenv()

True

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

response = llm.invoke([HumanMessage("잘 지냈어?")])
print(response.content)

네, 덕분에요! 저는 항상 새로운 정보를 배우고 업데이트하며 잘 지내고 있었어요.

관심 가져주셔서 감사해요. 당신은 어떻게 지내셨어요?

무엇이든 궁금한 점이나 도움이 필요하시면 편하게 말씀해주세요.
